# Observer Demobot

A minimal bot that connects to the eToro WebSocket, streams live prices, and logs
everything — without placing any trades. Useful for verifying connectivity and
inspecting the real-time price feed.

## Setup

1. Copy this notebook to `_demobot.ipynb` (gitignored) and fill in your credentials.
2. Run both cells — the bot will stream until you interrupt the kernel.

In [ ]:
"""Observer-only demobot — connects, streams prices, logs everything, trades nothing."""

import asyncio
import logging

from etoropy import EToroConfig, EToroTrading
from etoropy.models.websocket import WsInstrumentRate, WsPrivateEvent

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(name)s] %(levelname)s: %(message)s")
logger = logging.getLogger("demobot")

WATCH_SYMBOLS = ["AAPL", "BTC"]


class ObserverBot:
    def __init__(self, config: EToroConfig) -> None:
        self.etoro = EToroTrading(config=config)
        self.etoro.resolver.load_bundled_csv()
        self._running = False
        self._tick_count: dict[str, int] = {}

    async def start(self) -> None:
        logger.info("Starting observer bot (no trades will be placed)...")
        self._running = True

        self.etoro.on("price", self.on_price)
        self.etoro.on("order:update", self.on_order_update)
        self.etoro.on("error", self.on_error)

        await self.etoro.connect()
        logger.info("Connected.")

        self.etoro.subscribe_to_private_events()
        logger.info("Subscribed to private events.")

        await self.etoro.stream_prices(WATCH_SYMBOLS)
        logger.info("Streaming prices for: %s", WATCH_SYMBOLS)

        try:
            while self._running:
                await asyncio.sleep(1)
        except asyncio.CancelledError:
            pass
        finally:
            await self.stop()

    async def stop(self) -> None:
        self._running = False
        logger.info("Stopping observer bot...")
        await self.etoro.disconnect()
        logger.info("Stopped. Total ticks received: %s", self._tick_count)

    def on_price(self, symbol: str, instrument_id: int, rate: WsInstrumentRate) -> None:
        self._tick_count[symbol] = self._tick_count.get(symbol, 0) + 1
        mid = (rate.bid + rate.ask) / 2
        spread = rate.ask - rate.bid
        logger.info(
            "[tick #%d] %s  mid=%.4f  bid=%.4f  ask=%.4f  spread=%.4f",
            self._tick_count[symbol],
            symbol,
            mid,
            rate.bid,
            rate.ask,
            spread,
        )

    def on_order_update(self, event: WsPrivateEvent) -> None:
        logger.info(
            "Order event observed: OrderID=%d  Status=%d  InstrumentID=%d",
            event.order_id,
            event.status_id,
            event.instrument_id,
        )

    def on_error(self, error: Exception) -> None:
        logger.error("Error: %s", error)

In [ ]:
config = EToroConfig(
    api_key="YOUR_API_KEY",
    user_key="YOUR_USER_KEY",
    mode="demo",
    timeout=30.0,
    retry_attempts=3,
    retry_delay=1.0,
)

bot = ObserverBot(config=config)
await bot.start()